## Loss Function form Scratch

In [15]:
import numpy as np

<h4>Set of inputs

In [25]:
X = [[1, 2.2, 1.1, 5],     # sample of 4 inputs
     [0.1, 4, 0.9, 3], 
     [4, 2.1, 0.5, 3.2]]

y = np.array([0, 1, 1])

<h4>Function to create a layer

In [17]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):          
        # weights matrix is of n_inputs * n_neurons
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        # biases will of same size of neurons we have
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        # final output is going to be as (weights * inputs) + bias
        self.output = np.dot(inputs, self.weights) + self.biases

## ReLU Activation Function

In [18]:
class Activation_ReLU:
    def forward(self, inputs):
        # it helps in removing the negative values
        self.output = np.maximum(0, inputs)

#### As we removing -ve values in ReLU but this will cause the problem of dying neuron.
#### For -ve value it gives 0 so if we pass 0 to next neuron it becomes dead.
## Softmax Activation Function

In [19]:
class Activation_Softmax:
    def forward(self, inputs):
        # exponentiating the inputs to get rid off -ve values
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))
        # normalizing them to keep their meaning 
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)
        self.output = probabilities

<h4>Creating Layer 1 and applying ReLU Activation Function</h4>
This layer has 4 input features and 5 neurons

In [20]:
layer1 = Layer_Dense(4, 5)
activation1 = Activation_ReLU()
layer1.forward(X)
activation1.forward(layer1.output)
activation1.output

array([[0.        , 0.11265603, 0.        , 0.14914444, 0.        ],
       [0.        , 0.        , 0.        , 0.4312359 , 0.        ],
       [0.        , 0.        , 0.        , 0.24134714, 0.        ]])

## Creating Layer 2 and applying Softmax Activation Function 
<p>Output of Layer 1 is input to Layer 2</p>
So, it'll have 5 input and any number of neurons we can have.

In [21]:
layer2 = Layer_Dense(5, 2)
activation2 = Activation_Softmax()
layer2.forward(layer1.output)
layer2.output
activation2.forward(layer2.output)
activation2.output

array([[0.55970493, 0.44029507],
       [0.51868212, 0.48131788],
       [0.52722099, 0.47277901]])

### Loss Function - 
### Categorical-Cross-Entropy

In [22]:
class Loss:
    def calculate(self, y_hat, y):
        sample_loss = self.forward(y_hat, y)
        data_loss = np.mean(sample_loss)

        return data_loss

In [27]:
class Loss_Categorical_Cross_Entropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        # bound the predicted values to a specific range to handle numerical stability concerns
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # if scaler target value
        if len(y_true.shape) == 1:
            correct_confidence = y_pred_clipped[range(samples), y_true]
        # if one hot encodded target value
        elif len(y_true.shape) == 2:
            correct_confidence = np.sum(y_pred_clipped * y_true, axis = 1)

        negative_log_likelihood = -np.log(correct_confidence)

        return negative_log_likelihood

In [28]:
loss_function = Loss_Categorical_Cross_Entropy()
loss = loss_function.calculate(activation2.output, y)
print("Loss : ", loss)

Loss :  0.6869000387125329


### Loss for this sample is : 0.6869
<h4>On the basis of calculates loss the backpropagation is performed.</h4>
<p>If loss is very high then the weigth can be adjusted by larger values as we are far from target.</p> 
<p>If loss is very low then the weight are adjusted by smaller values as we are nearer to target.